In [ ]:
from mytransforms import PasteImage
from torchvision import transforms as T
import cv2
import torch
from mydisplay import mydisplay

In [ ]:
from yolomodel import MyTinyYolo
model = MyTinyYolo()
model.load()
model.eval();

In [ ]:
from converter import Converter
converter = Converter(13, 13, 20, 5, model)

In [ ]:
preprocess = T.Compose([
    T.ToTensor(),
    T.Resize(416),
    PasteImage(416) # somewhat surprising that the input has to be the same size as training, otherwise it's pretty bad
])

In [ ]:
vid = cv2.VideoCapture(0)
while True:
    ret, frame = vid.read()

    img = preprocess(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    with torch.no_grad():
        grid_output = model(img.unsqueeze(dim=0)).squeeze(dim=0)
    bbox_output, class_output, p_output = converter.grid_to_bbox(grid_output, cmin=0.5)
    bbox_sv, class_sv, p_sv = converter.nonmax_compressed(bbox_output, class_output, p_output)

    for bbox, class_name, p in zip(bbox_sv, class_sv, p_sv):
        H, W = img.shape[-2:]
        frame_H, frame_W = frame.shape[:2]
        hx, hy, hw, hh = bbox
        hh = frame_W / frame_H * hh
        hy = (hy - 0.5) * frame_W / frame_H + 0.5
        xmin = int((hx - 0.5 * hw) * frame_W)
        xmin = max(0, xmin)
        xmax = int((hx + 0.5 * hw) * frame_W)
        xmax = min(frame_W, xmax)
        ymin = int((hy - 0.5 * hh) * frame_H)
        ymin = max(0, ymin)
        ymax = int((hy + 0.5 * hh) * frame_H)
        ymax = min(frame_W, ymax)
        if class_name == 'cat':
            color = (0,255,0)
        elif class_name == 'person':
            color = (255,0,0)
        else:
            color = (255,255,255)
        cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), color=color, thickness=5)
        text = "{0:s} {1:5.3f}".format(class_name, p)
        cv2.putText(frame, text, (xmin,int(ymin*1.1)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 5)
    cv2.imshow('mycamera', frame)

    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows() 
        vid.release()
        break

In [ ]:
# show the last frame to double check
mydisplay(img, bbox_sv, class_sv, p_sv)